In [35]:
def bytes_info(size): #выводит размер файла в байтах, килобайтах и мегабайтах
    print('Размер файла:',  size, 'байт или ', round(size/1024, 3), 'Кб или ', round(size/1024**2, 3), 'Мб')

def compare_size(old, new): #выводит процент, который составляет новый размер файла от старого
    print('Новый размер файла составляет ', round(new/old, 3)*100, '% от старого')

In [36]:
# загружаем текст и вычисляем его исходный размер
with open('../text_for_decap.txt', 'r', encoding='ascii') as f:
        text = f.read()

main_size=len(bytearray(text, 'ascii'))
bytes_info(main_size)

text1=text
text2=text.lower()
with open('text_decape.txt', 'w', encoding='ascii') as f:
        f.write(text2)
text3=text1[::-1]
with open('text_for_decap_reverse.txt', 'w', encoding='ascii') as f:
        f.write(text3)
text4=text2[::-1]
with open('text_decap_reverse.txt', 'w', encoding='ascii') as f:
        f.write(text4)


Размер файла: 3171529 байт или  3097.196 Кб или  3.025 Мб


In [37]:
import sys
from collections import defaultdict

# === arithmeticcoding.py (упрощённая версия) ===
class ArithmeticCoderBase:
    def __init__(self, numbits):
        if numbits < 1:
            raise ValueError("State size out of range")
        self.num_state_bits = numbits
        self.full_range = 1 << self.num_state_bits
        self.half_range = self.full_range >> 1
        self.quarter_range = self.half_range >> 1
        self.minimum_range = self.quarter_range + 2
        self.state_mask = self.full_range - 1
        self.low = 0
        self.high = self.state_mask


class ArithmeticEncoder(ArithmeticCoderBase):
    def __init__(self, numbits, bitout):
        super().__init__(numbits)
        self.output = bitout
        self.num_underflow = 0

    def write(self, freqs, symbol):
        self.update(freqs, symbol)

    def finish(self):
        self.output.write(1)
        for _ in range(self.num_underflow):
            self.output.write(0)

    def update(self, freqs, symbol):
        low = self.low
        high = self.high
        range_val = high - low + 1
        total = freqs.get_total()
        symlow = freqs.get_low(symbol)
        symhigh = freqs.get_high(symbol)
        newlow = low + symlow * range_val // total
        newhigh = low + symhigh * range_val // total - 1
        self.low, self.high = newlow, newhigh
        while ((self.low ^ self.high) & self.half_range) == 0:
            self.shift()
            self.low = ((self.low << 1) & self.state_mask)
            self.high = ((self.high << 1) | 1) & self.state_mask
        while (self.low & ~self.high & self.quarter_range) != 0:
            self.underflow()
            self.low = (self.low << 1) ^ self.half_range
            self.high = ((self.high ^ self.half_range) << 1) | self.half_range | 1

    def shift(self):
        bit = self.low >> (self.num_state_bits - 1)
        self.output.write(bit)
        for _ in range(self.num_underflow):
            self.output.write(bit ^ 1)
        self.num_underflow = 0

    def underflow(self):
        self.num_underflow += 1


class ArithmeticDecoder(ArithmeticCoderBase):
    def __init__(self, numbits, bitin):
        super().__init__(numbits)
        self.input = bitin
        self.code = 0
        for _ in range(self.num_state_bits):
            self.code = self.code << 1 | self.read_code_bit()

    def read(self, freqs):
        total = freqs.get_total()
        range_val = self.high - self.low + 1
        offset = self.code - self.low
        value = ((offset + 1) * total - 1) // range_val
        start = 0
        end = freqs.get_symbol_limit()
        while end - start > 1:
            mid = (start + end) >> 1
            if freqs.get_low(mid) > value:
                end = mid
            else:
                start = mid
        symbol = start
        self.update(freqs, symbol)
        return symbol

    def shift(self):
        self.code = ((self.code << 1) & self.state_mask) | self.read_code_bit()

    def underflow(self):
        self.code = (self.code & self.half_range) | ((self.code << 1) & (self.state_mask >> 1)) | self.read_code_bit()

    def read_code_bit(self):
        temp = self.input.read()
        return temp if temp != -1 else 0

    def update(self, freqs, symbol):
        low = self.low
        high = self.high
        range_val = high - low + 1
        symlow = freqs.get_low(symbol)
        symhigh = freqs.get_high(symbol)
        newlow = low + symlow * range_val // freqs.get_total()
        newhigh = low + symhigh * range_val // freqs.get_total() - 1
        self.low = newlow
        self.high = newhigh
        while ((self.low ^ self.high) & self.half_range) == 0:
            self.shift()
            self.low = ((self.low << 1) & self.state_mask)
            self.high = ((self.high << 1) | 1) & self.state_mask
        while (self.low & ~self.high & self.quarter_range) != 0:
            self.underflow()
            self.low = (self.low << 1) ^ self.half_range
            self.high = ((self.high ^ self.half_range) << 1) | self.half_range | 1


class FrequencyTable:
    def get_symbol_limit(self): raise NotImplementedError()
    def get(self, symbol): raise NotImplementedError()
    def set(self, symbol, freq): raise NotImplementedError()
    def increment(self, symbol): raise NotImplementedError()
    def get_total(self): raise NotImplementedError()
    def get_low(self, symbol): raise NotImplementedError()
    def get_high(self, symbol): raise NotImplementedError()


class SimpleFrequencyTable(FrequencyTable):
    def __init__(self, freqs):
        self.frequencies = list(freqs)
        self.total = sum(self.frequencies)
        self.cumulative = None

    def get_symbol_limit(self): return len(self.frequencies)

    def get(self, symbol): return self.frequencies[symbol]

    def set(self, symbol, freq):
        self.total += freq - self.frequencies[symbol]
        self.frequencies[symbol] = freq
        self.cumulative = None

    def increment(self, symbol):
        self.frequencies[symbol] += 1
        self.total += 1
        self.cumulative = None

    def get_total(self): return self.total

    def get_low(self, symbol):
        self._init_cumulative()
        return self.cumulative[symbol]

    def get_high(self, symbol):
        self._init_cumulative()
        return self.cumulative[symbol + 1]

    def _init_cumulative(self):
        if self.cumulative is None:
            cumul = [0]
            s = 0
            for f in self.frequencies:
                s += f
                cumul.append(s)
            self.cumulative = cumul


class FlatFrequencyTable(FrequencyTable):
    def __init__(self, numsyms):
        if numsyms < 1:
            raise ValueError()
        self.numsymbols = numsyms

    def get_symbol_limit(self): return self.numsymbols

    def get(self, symbol): return 1

    def get_total(self): return self.numsymbols

    def get_low(self, symbol): return symbol

    def get_high(self, symbol): return symbol + 1


# === ppmmodel.py ===
class PpmModel:
    def __init__(self, order, symbollimit, escapesymbol):
        self.model_order = order
        self.symbol_limit = symbollimit
        self.escape_symbol = escapesymbol
        if order >= 0:
            self.root_context = self.Context(symbollimit, order >= 1)
            self.root_context.frequencies.increment(escapesymbol)
        else:
            self.root_context = None
        self.order_minus1_freqs = FlatFrequencyTable(symbollimit)

    def increment_contexts(self, history, symbol):
        if self.model_order == -1:
            return
        ctx = self.root_context
        ctx.count += 1
        ctx.frequencies.increment(symbol)
        for i, sym in enumerate(history[:self.model_order]):
            subctxs = ctx.subcontexts
            assert subctxs is not None
            if subctxs[sym] is None:
                subctxs[sym] = self.Context(self.symbol_limit, i + 1 < self.model_order)
                subctxs[sym].frequencies.increment(self.escape_symbol)
            ctx = subctxs[sym]
            ctx.count += 1
            ctx.frequencies.increment(symbol)

    class Context:
        def __init__(self, symbols, hassubctx):
            self.frequencies = SimpleFrequencyTable([0] * symbols)
            self.subcontexts = ([None] * symbols) if hassubctx else None
            self.count = 0


# === Bit stream classes ===
class DummyBitOutputStream:
    def __init__(self): self.bits = []

    def write(self, bit): self.bits.append(bit)

    def get_bits(self): return self.bits


class DummyBitInputStream:
    def __init__(self, bits):
        self.bits = bits
        self.index = 0

    def read(self):
        if self.index >= len(self.bits):
            return -1
        bit = self.bits[self.index]
        self.index += 1
        return bit


# === PPM Encoder / Decoder ===
def encode_symbol(model, history, symbol, encoder, method='A'):
    for order in reversed(range(len(history) + 1)):
        ctx = model.root_context
        for sym in history[:order]:
            assert ctx.subcontexts is not None
            ctx = ctx.subcontexts[sym]
            if ctx is None:
                break
        else:
            # Проверяем наличие символа в текущем контексте
            if symbol != 256 and ctx.frequencies.get(symbol) > 0:
                encoder.write(ctx.frequencies, symbol)
                return

            # Рассчитываем escape-вероятность
            c = ctx.count
            symbol_limit = ctx.frequencies.get_symbol_limit()
            unique_symbols = 0
            once_count = 0
            for i in range(symbol_limit):
                freq = ctx.frequencies.get(i)
                if freq > 0:
                    unique_symbols += 1
                    if freq == 1:
                        once_count += 1
            m1 = once_count
            m = unique_symbols

            escape_prob = {
                'A': 1 / (c + 1),
                'B': m / max(c, 1),
                'C': m / (m + c),
                'D': m / (2 * max(c, 1)),
                'X': max(m1 / max(c, 1), sys.float_info.min)
            }[method]

            escape_freq_int = max(1, int(round(ctx.frequencies.get_total() * escape_prob)))

            # Обновляем частоту escape, если он уже есть
            if ctx.frequencies.get(256) > 0:
                ctx.frequencies.set(256, escape_freq_int)
            else:
                # Иначе добавляем его
                freq_list = list(ctx.frequencies.frequencies) + [escape_freq_int]
                ctx.frequencies = SimpleFrequencyTable(freq_list)

            encoder.write(ctx.frequencies, 256)
            return

    encoder.write(model.order_minus1_freqs, symbol)


def decode_symbol(model, history, decoder, method='A'):
    for order in reversed(range(len(history) + 1)):
        ctx = model.root_context
        for sym in history[:order]:
            assert ctx.subcontexts is not None
            ctx = ctx.subcontexts[sym]
            if ctx is None:
                break
        else:
            if ctx.frequencies.get_total() > 0:
                # Рассчитываем escape-вероятность
                c = ctx.count
                symbol_limit = ctx.frequencies.get_symbol_limit()
                unique_symbols = 0
                once_count = 0
                for i in range(symbol_limit):
                    freq = ctx.frequencies.get(i)
                    if freq > 0:
                        unique_symbols += 1
                        if freq == 1:
                            once_count += 1
                m1 = once_count
                m = unique_symbols

                escape_prob = {
                    'A': 1 / (c + 1),
                    'B': m / max(c, 1),
                    'C': m / (m + c),
                    'D': m / (2 * max(c, 1)),
                    'X': max(m1 / max(c, 1), sys.float_info.min)
                }[method]

                escape_freq_int = max(1, int(round(ctx.frequencies.get_total() * escape_prob)))

                # Обновляем или добавляем escape
                if ctx.frequencies.get(256) > 0:
                    ctx.frequencies.set(256, escape_freq_int)
                else:
                    freq_list = list(ctx.frequencies.frequencies) + [escape_freq_int]
                    ctx.frequencies = SimpleFrequencyTable(freq_list)

                symbol = decoder.read(ctx.frequencies)
                if symbol != 256:
                    return symbol
                else:
                    continue

    return decoder.read(model.order_minus1_freqs)


# === Main functions ===
def compress_data(data, method='A', is_text=True):
    if is_text:
        input_bytes = list(data.encode("ascii")) + [256]
    else:
        input_bytes = data + [256]

    model = PpmModel(MODEL_ORDER, 257, 256)
    bitout = DummyBitOutputStream()
    enc = ArithmeticEncoder(32, bitout)
    history = []

    for symbol in input_bytes:
        encode_symbol(model, history, symbol, enc, method=method)
        model.increment_contexts(history, symbol)
        if model.model_order >= 1:
            if len(history) == model.model_order:
                history.pop()
            history.insert(0, symbol)

    enc.finish()
    return bitout.get_bits()


def decompress_data(bit_data, method='A', output_type="text"):
    bitin = DummyBitInputStream(bit_data)
    dec = ArithmeticDecoder(32, bitin)
    model = PpmModel(MODEL_ORDER, 257, 256)
    history = []
    result = []

    while True:
        symbol = decode_symbol(model, history, dec, method=method)
        if symbol == 256:
            break
        result.append(symbol)
        model.increment_contexts(history, symbol)
        if model.model_order >= 1:
            if len(history) == model.model_order:
                history.pop()
            history.insert(0, symbol)

    if output_type == "text":
        try:
            return bytes(result).decode("ascii")
        except UnicodeDecodeError:
            return bytes(result).decode("latin1")
    elif output_type == "list":
        return result
    else:
        raise ValueError("output_type должен быть 'text' или 'list'")


# === Main test ===
# if __name__ == "__main__":
#     MODEL_ORDER = 3

#     # Тест со строкой
#     original_text = "abracadabra"
#     print("Original Text:", original_text)
#     compressed = compress_data(original_text, method='A', is_text=True)
#     print("Compressed Bits:", compressed)
#     print("Size:", len(compressed) / 8, "bytes")
#     # decoded = decompress_data(compressed, method='C', output_type='text')
#     # print("Decoded Text:", decoded)

#     # Тест с числами
#     original_numbers = [10, 20, 30, 40, 50, 10, 20, 30]
#     print("\nOriginal Numbers:", original_numbers)
#     compressed = compress_data(original_numbers, method='A', is_text=False)
#     print("Compressed Bits:", compressed)
#     print("Size:", len(compressed) / 8, "bytes")
#     # decoded = decompress_data(compressed, method='A', output_type='list')
#     # print("Decoded Numbers:", decoded)

In [38]:
with open("Results_text.txt", "w") as file:
    file.write('Результаты работы PPM\n')
m={}
bestsizes={}
i=1
for text in [text1, text2, text3, text4]:
    with open("Results_text.txt", "a") as file:
        file.write(f"\n\nИсходный текст: {text[:100]}...\nДлина: {len(text)} символов\nИсходный размер: {main_size} байт")
    m.clear()

    # Сжимаем с разными методами ухода
    for method in ['A', 'B', 'C', 'D', 'X']:
            # Сжатие
        compressed = compress_data(text, method)
            # print(compressed)
            # print(f"Сжатые данные: {len(compressed)} байт ({len(compressed)/main_size*100:.2f}% от исходного)")
        m[method]= len(compressed)/8
        with open("Results_text.txt", "a") as file:
            file.write(f"\n\nМетод ухода: {method}\nСжатые данные: {len(compressed)/8} байт ({(len(compressed)/8)/main_size*100:.2f}% от исходного)")
    
    with open("Results_text.txt", "a") as file:
        min_value = min(m.values())
        min_keys = [k for k, v in m.items() if v == min_value]
        bestsizes[i]={min(m, key=m.get):min_value}
        # bestsizes.append(min_value)
        if len(min_keys)==1:
            file.write(f"\n\nВывод по методу сжатия: Наилучший результат показал метод оценки вероятности ухода {min(m, key=m.get)} - {min_value} байт")
        else:
            file.write(f"\n\nВывод по методу сжатия: Наилучший результат показали методы оценки вероятности ухода {', '.join(map(str, min_keys))} - {min_value} байт")
    
    i+=1

        

Дополнительная кодировка массива исключения для декапитализированного текста

In [39]:
import re
def check_caps(t): # Поиск всех слов с заглавными буквами
    pattern = r'\b\w*[A-ZА-ЯЁ]\w*\b'
    # Поиск всех совпадений
    words_with_capital = re.findall(pattern, t)
    return words_with_capital

def cap_symb(t): # Поиск всех символов, являющихся заглавными буквами
    caps=0
    for char in t:
        if char.isupper():
            caps+=1
    return caps

In [40]:
def generate_bit_array(text, extra): # Применение общих правил декапитализации и создание битового массива, в котором 1 указывают на позиции исключений
    bit_array = []
    text_list = list(text)  # Преобразуем строку в список для изменения символов
    # print(text)
    expected_lowers=[True]*len(text)
    for i, char in enumerate(text):
        if char.isalpha():
            
            expected_lower = True  # по умолчанию ожидаем строчную
            
            if extra:
                # Проверка на название (2 заглавние подряд)
                if i>=2 and ((text[i - 2].isalpha() and text[i-2].isupper()) and (text[i-1].isalpha() and text[i-1].isupper())):
                    expected_lower = False
                    text_list[i]=text_list[i].lower()

            # Проверка правил
            if i == 0 or (i > 1 and text[i - 2:i] in {". ", "! ", "? ", "\n"}) or (i > 1 and text[i - 1:i] in {'"'}):
                expected_lower = False
                text_list[i]=text_list[i].lower()
                # caps.append(i)

            # Проверка местоимения 'I'
            if char == 'I' and (i == 0 or not text[i - 1].isalpha()) and (
                    i == len(text) - 1 or not text[i + 1].isalpha()):
                expected_lower = False
                text_list[i]=text_list[i].lower()
                # caps.append(i)

            # Проверка исключений
            if (char.islower() and not expected_lower) or (char.isupper() and expected_lower):
                bit_array.append(1)
                # caps.append(i)
            else:
                bit_array.append(0)

            expected_lowers[i]=expected_lower
        else:
            bit_array.append(0)  # не буквы игнорируем

    text_new = ''.join(text_list)
    return bit_array, text_new

In [41]:
rules, text_decap=generate_bit_array(text1, True)
print('Исходный размер массива исключений:')
# print(rules[:100])
size_of_exceptions=len(rules)/8 # в байтах
# print(len(rules))
bytes_info(size_of_exceptions)

# закодируем количество нулей между 1
numbers=[]
z=0
for i in rules:
    if i==0:
        z+=1
    else:
        numbers.append(z)
        z=0
# print("Массив растояний между исключениями:", numbers)

intt=list(map(str, numbers))
string_int=','.join(intt)
print(string_int[:100], '...')

Исходный размер массива исключений:
Размер файла: 396441.125 байт или  387.15 Кб или  0.378 Мб
1,7,0,6,0,3,0,6,1,0,3,0,2,0,10,3,5,8,10,8,8,10,9,5,80,535,13,323,7,215,152,19,34,147,26,670,31,17,7, ...


In [42]:
import struct
with open("Results_exceptions.txt", "w") as file:
    file.write('Результаты работы PPM\n')
m={}
data=len(struct.pack(f'{len(numbers)}q', *numbers))
with open("Results_exceptions.txt", "a") as file:
    file.write(f"\n\nИсходный текст: {numbers[:100]}...\nДлина: {len(numbers)} символов\nИсходный размер: {data} байт")
m.clear()

    # Сжимаем с разными методами ухода
for method in ['A', 'B', 'C', 'D', 'X']:    
    print(f"\nМетод ухода: {method}")
            # Сжатие
    compressed = compress_data(string_int, method)
        # print(compressed)compress_bytes
    print(f"Сжатые данные: {len(compressed)} байт ({len(compressed)/data*100:.2f}% от исходного)")
    m[method]= len(compressed)
    with open("Results_exceptions.txt", "a") as file:
        file.write(f"\n\nМетод ухода: {method}\nСжатые данные: {len(compressed)} байт ({len(compressed)/data*100:.2f}% от исходного)")
    
with open("Results_exceptions.txt", "a") as file:
    min_value = min(m.values())
    min_keys = [k for k, v in m.items() if v == min_value]
    if len(min_keys)==1:
        file.write(f"\n\nВывод по методу сжатия: Наилучший результат показал метод оценки вероятности ухода {min(m, key=m.get)} - {min_value} байт")
    else:
        file.write(f"\n\nВывод по методу сжатия: Наилучший результат показали методы оценки вероятности ухода {', '.join(map(str, min_keys))} - {min_value} байт")
        

        


Метод ухода: A
Сжатые данные: 246678 байт (90.20% от исходного)

Метод ухода: B
Сжатые данные: 256195 байт (93.68% от исходного)

Метод ухода: C
Сжатые данные: 250328 байт (91.54% от исходного)

Метод ухода: D
Сжатые данные: 247853 байт (90.63% от исходного)

Метод ухода: X
Сжатые данные: 246972 байт (90.31% от исходного)


In [45]:
with open("Results_text_decap.txt", "w") as file:
    file.write('Результаты работы PPM\n')

    file.write(f"\n\nВывод по методу сжатия (text2): Наилучший результат показал метод оценки вероятности ухода {list(bestsizes[2].keys())[0]} - {list(bestsizes[2].values())[0]+m['A']} байт")
    file.write(f"\n\nМетод ухода: {method}\nСжатые данные: {list(bestsizes[2].values())[0]+m['A']} байт ({(list(bestsizes[2].values())[0]+m['A'])/main_size*100:.2f}% от исходного)")

    file.write(f"\n\nВывод по методу сжатия (text4): Наилучший результат показал метод оценки вероятности ухода {list(bestsizes[4].keys())[0]} - {list(bestsizes[4].values())[0]+m['A']} байт")       
    file.write(f"\n\nМетод ухода: {method}\nСжатые данные: {list(bestsizes[4].values())[0]+m['A']} байт ({(list(bestsizes[4].values())[0]+m['A'])/main_size*100:.2f}% от исходного)")